In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, LeaveOneOut
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.metrics import  make_scorer, accuracy_score, f1_score, confusion_matrix, roc_auc_score, silhouette_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics.cluster import normalized_mutual_info_score
import pickle
import joblib

In [2]:
!where python

c:\Users\polyx\Desktop\Pattern-Recognition-Gr7\venv\Scripts\python.exe
C:\Users\polyx\AppData\Local\Programs\Python\Python310\python.exe
C:\Users\polyx\AppData\Local\Microsoft\WindowsApps\python.exe


In [17]:
import sklearn
print(np.__version__)

2.1.2


In [21]:
with open('LeaveOneOut3_grid_search.pkl', 'rb') as file:
    grid_search_results = pickle.load(file)

print(grid_search_results.cv_results_)

{'mean_fit_time': array([  2.59240388,   4.94892021,  12.23856829,   2.577815  ,
         5.00629046,  12.3384821 ,   2.57122662,   5.01834147,
        12.34285698,   0.18154431,   0.2497929 ,   0.24886702,
         0.25186472,   0.25162556,   0.25379781, 143.70549689,
       140.20143581, 103.2431931 ,  42.90777675,  33.40542629,
        36.47155888]), 'std_fit_time': array([2.25090048e-01, 3.20644571e-01, 7.79491934e-01, 2.01828013e-01,
       3.75159001e-01, 8.80467386e-01, 2.09982630e-01, 3.85550539e-01,
       8.72356519e-01, 2.86780108e-02, 2.92690872e-02, 2.19167129e-02,
       2.52149854e-02, 2.70853687e-02, 2.39375775e-02, 4.36284000e+01,
       3.87216349e+01, 3.72067368e+01, 1.58860180e+01, 1.16222886e+01,
       1.06904449e+01]), 'mean_score_time': array([0.08705557, 0.08968062, 0.10078637, 0.08731133, 0.09039824,
       0.10098086, 0.08675533, 0.09079176, 0.10092208, 0.11090029,
       0.11369007, 0.11186075, 0.11098885, 0.112309  , 0.11234091,
       0.12897029, 0.1409933

In [22]:
def create_ranked_lists(grid_search_results):

    cv_results = grid_search_results.cv_results_
    
    df = pd.DataFrame({
        'mean_fit_time': cv_results['mean_fit_time'],
        'mean_score_time': cv_results['mean_score_time'],
        'mean_test_f1': cv_results['mean_test_f1'],
        'params': cv_results['params']  
    })

    # Extract F1 scores
    f1_scores = cv_results.get('mean_test_f1', None)
    
    if f1_scores is not None:
        # Count how many have a perfect F1 score 
        perfect_f1_count = sum(1 for score in f1_scores if score == 1.0)
        print(f"Number of models with a perfect F1 score: {perfect_f1_count}")


    # Filter out models that dont have perfect f1 score
    df_perfect_f1 = df[df['mean_test_f1'] == 1.0]



    # Rank models based on mean_score_time
    df_ranked_by_score_time = df_perfect_f1.sort_values(by='mean_score_time', ascending=True).reset_index(drop=True)

    # Rank models based on mean_fit_time
    df_ranked_by_fit_time = df_perfect_f1.sort_values(by='mean_fit_time', ascending=True).reset_index(drop=True)

    # Rank models based on the combined time
    df_perfect_f1['combined_time'] = df_perfect_f1['mean_score_time'] + df_perfect_f1['mean_fit_time']
    df_ranked_by_combined_time = df_perfect_f1.sort_values(by='combined_time', ascending=True).reset_index(drop=True)


    return df_ranked_by_score_time, df_ranked_by_fit_time, df_ranked_by_combined_time

In [23]:
ranked_by_score_time, ranked_by_fit_time, ranked_by_combined_time = create_ranked_lists(grid_search_results)

# Print Ranked by Mean Score Time
print("Ranked by Mean Score Time:")
print(ranked_by_score_time[['mean_score_time', 'mean_test_f1', 'params']])

# Print Ranked by Mean Fit Time
print("\nRanked by Mean Fit Time:")
print(ranked_by_fit_time[['mean_fit_time', 'mean_test_f1', 'params']])

# Print Ranked by Combined Time (mean_score_time + mean_fit_time)
print("\nRanked by Combined Time:")
print(ranked_by_combined_time[['combined_time', 'mean_test_f1', 'params']])

Number of models with a perfect F1 score: 6
Ranked by Mean Score Time:
   mean_score_time  mean_test_f1  \
0         0.103748           1.0   
1         0.107972           1.0   
2         0.110989           1.0   
3         0.111861           1.0   
4         0.112309           1.0   
5         0.112341           1.0   

                                              params  
0  {'classifier': SVC(probability=True, random_st...  
1  {'classifier': SVC(probability=True, random_st...  
2  {'classifier': KNeighborsClassifier(), 'classi...  
3  {'classifier': KNeighborsClassifier(), 'classi...  
4  {'classifier': KNeighborsClassifier(), 'classi...  
5  {'classifier': KNeighborsClassifier(), 'classi...  

Ranked by Mean Fit Time:
   mean_fit_time  mean_test_f1  \
0       0.248867           1.0   
1       0.251626           1.0   
2       0.251865           1.0   
3       0.253798           1.0   
4      33.405426           1.0   
5      36.471559           1.0   

                          

C:\Users\polyx\AppData\Local\Temp\ipykernel_35648\3384443238.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_perfect_f1['combined_time'] = df_perfect_f1['mean_score_time'] + df_perfect_f1['mean_fit_time']


In [24]:
ranked_by_combined_time[['params'][0]][0]

{'classifier': KNeighborsClassifier(),
 'classifier__n_neighbors': 5,
 'classifier__weights': 'uniform',
 'feature_selection': 'passthrough'}